In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sys
import cPickle as pickle
import html5lib
import os
import ipyparallel as ipp

from bs4 import BeautifulSoup as bs
from urllib2 import urlopen
from urllib2 import Request
from urllib2 import URLError

reload(sys)
sys.setdefaultencoding('utf-8')
sys.setrecursionlimit(50000)

%matplotlib inline

In [2]:
#Corremos el cluster
rc = ipp.Client()
ar = rc[:].apply_async(os.getpid)
pid_map = ar.get_dict()

# Cargamos los datos de las propiedades

In [ ]:
#Cargo toda la info
propertyD = pd.read_csv('../../../../datos/properati-AR-2017-08-01-properties-sell.csv')
propertyData = propertyD[(propertyD['price_aprox_usd'].isnull() == False)]

In [4]:
propertyData.info()
pickle.dump(propertyData,open("save.p","wb"))
#Todas tienen la url.

# Completamos los cuartos con las descripciones.

In [4]:
caracteristicas = propertyData[['id','description','title','properati_url','rooms']]
caracteristicas.head(10)

,id,description,title,properati_url,rooms
0,c8a66ee9e3da083ab66dacd165e8a5cc718b1cf2,CASA CON LOCAL EN VENTA EN PADUA.2 DORMITORIOS...,CASA EN VENTA,http://www.properati.com.ar/udh_venta_casa_mer...,NaN
1,a205c53378e7879cf7519659317fb410c060d20e,CHALET VENTA 4 AMBIENTES PADUA.EXCELENTE CHALE...,CASA EN VENTA,http://www.properati.com.ar/20hi_venta_casa_me...,NaN
4,9446052c949050359fb08b9f1ba83ba2eaa20c99,Casa Multifamiliar en Mataderos!!! Sobre lote ...,CASA EN VENTA,http://www.properati.com.ar/4m4m_venta_casa_ma...,5.0
5,4d3b492ed5f9c74cdf33b56b98328e5c950e3f9c,"Venta - Departamento - Argentina, Buenos Aires...",Departamento en Venta ubicado en Arcos de Pilar,http://www.properati.com.ar/518a_venta_departa...,2.0
6,535441f569473f6e5d687c38b27d9b9c6e69e284,"Venta - Local - Argentina, Buenos Aires, Zona ...",Local en Venta ubicado en San Isidro,http://www.properati.com.ar/52p4_venta_local_s...,1.0
8,27d87ac71e7c00afa9fa7428b943ec263ff68bb4,"Venta - Casa - Argentina, Patagonia, San Marti...",Casa en Venta ubicado en Quilquihue,http://www.properati.com.ar/568j_venta_casa_sa...,4.0
9,df77a06585671e637d2d86f8d22b70d61cfecb18,"Venta - Casa - Argentina, Patagonia, San Marti...",Casa en Venta ubicado en Rio Hermoso,http://www.properati.com.ar/568m_venta_casa_sa...,3.0
10,61f3789618edabeaa7b5fc06ff36112f8a06557f,"Venta - Departamento - Argentina, Buenos Aires...",Departamento en Venta ubicado en Pilar,http://www.properati.com.ar/59ec_venta_departa...,2.0
11,c7ac98ce42a2689a8aee59fb1e91c6b7194c033c,"Venta - Casa - Argentina, Patagonia, San Marti...",Casa en Venta ubicado en Centro,http://www.properati.com.ar/5b9n_venta_casa_sa...,4.0
12,e842ed22eac89b9bffff146377abfd5bceb0b0cd,"Venta - Casa - Argentina, Buenos Aires, Zona N...",Casa en Venta ubicado en Benavidez,http://www.properati.com.ar/5paq_venta_casa_be...,6.0


In [5]:
#caracteristicas['rooms'].isnull().sum()
caracteristicas = caracteristicas[140000:]

In [6]:
def chequearCuartos(x):
    if(not pd.isnull(x[4])):
        return x[4]
    return buscarCuartos(x[1]) or buscarCuartos(x[2])
        
def buscarCuartos(x):
    
    caract = ""
    caractViviendas = ['dorm','hab','cuarto'] 
    
    if( x == None or x == ""):
        return None
    
    if type( x ) is str:
        x = x.lower()
    if type( x ) is float:
        x = str(x).lower()
    #Queda una lista separada por comas.    
    x = x.split()
    
    for j in xrange(len(caractViviendas)):
        
        aux = ""
        palabraAct = caractViviendas[j]
        
        if( palabraAct in x ):
            indice = x.index( palabraAct )
            #caract = buscarCantidadDorm(x,indice)
            caract = x[indice-1]
            break;
            
        else:
            for ind in xrange( len(x) ):
                if( palabraAct in x[ind] ):
                    #caract = buscarCantidadDorm(x,ind)
                    caract = x[ind-1]
                    break;
    if(caract.isdigit()):
        return int(caract)
    return None  

In [7]:
caracteristicas['rooms'] = caracteristicas.apply(chequearCuartos, axis=1)

In [42]:
#caracteristicas['rooms'].isnull().sum()#62257

1

In [8]:
caracteristicas.head(10)

,id,description,title,properati_url,rooms
168728,77816f8811f96453c3a990de643fcf2d2659ac1b,SE TRATA DE UNA VIVIENDA MULTI FAMILIAR CONSIS...,Casa - Merlo,http://www.properati.com.ar/1bqtk_venta_casa_m...,NaN
168730,bed12005637a003d531a4dcc3aa5a1076787d3b1,Impecable dpto 2 amb. a estrenar con detalles ...,Departamento - San Fernando,http://www.properati.com.ar/1bqto_venta_depart...,NaN
168731,fb0007abeb74caed5c693783b617a1d1befa82df,Sergio Villella Inmobiliaria presenta departam...,Ov. Lagos 600 - Venta departamento monoambiente.,http://www.properati.com.ar/1bqtq_venta_depart...,NaN
168733,877ca74c858bc85dd3373eb265795c7fc3afc1e7,"LINDO DUPLEX DE 4 AMB: 3 DORM AMPLIOS, 2 BAÑOS...",Casa en Florida Belgrano-Oeste,http://www.properati.com.ar/1bqts_venta_casa_f...,3.0
168734,ba7121c00740ba13447d21cda96286cead1615b7,PH AL FRENTE CON ENTRADA INDEPENDIENTE Y ESPAC...,PH 3 AMB. APTO CRÉDITO - Moron,http://www.properati.com.ar/1bqtt_venta_ph_mor...,4.0
168735,997b2405ded6af49f75ebe01ff168cfb82291223,Departamento de 2 ambientes a estrenar en comp...,Departamento - Condominios Intercontinental,http://www.properati.com.ar/1bqtu_venta_depart...,NaN
168736,53162225dcf5fe8202e46f95aad3d6183f8cfe9d,IMPORTANTE PROPIEDAD DESARROLLADA EN SOLO UNA ...,Casa - La Plata,http://www.properati.com.ar/1bqtv_venta_casa_l...,NaN
168737,31dcf5ef4a9af6f1803aa8b998c9be95b223871f,Lindísimo departamento a Estrenar: está ubic...,BELLISIMO 2 AMBIENTES CON GRAN TERRAZA Y COCHERA.,http://www.properati.com.ar/1bqtx_venta_depart...,NaN
168738,1bcd00c5319313fd70463337fce4d89bb73ebbbc,Winterra S.A.Catamarca 1150CMCPSI5015,Departamento Lagoon Pilar Sector Serena!,http://www.properati.com.ar/1bqty_venta_depart...,NaN
168739,08acdc231bd21c964a16a8af510c4bd444bb3f77,"Casa de 1 dormitorio, cocina comedor y baño. T...",Casa - El Bolson,http://www.properati.com.ar/1bqu0_venta_casa_s...,1.0


# Ahora añadimos los ambientes

In [9]:
caracteristicas['ambientes'] = pd.Series(None, index=caracteristicas.index)

In [10]:
def chequearAmbientes(x):
    
    if(not pd.isnull(x[5])):
        return x[5]
    return buscarAmbientes(x[1]) or buscarAmbientes(x[2])
        
def buscarAmbientes(x):
    
    caract = ""
    caractViviendas = ['amb','monoamb'] 
    
    if( x == None or x == ""):
        return None
    
    if type( x ) is str:
        x = x.lower()
    if type( x ) is float:
        x = str(x).lower()
    #Queda una lista separada por comas.    
    x = x.split()
    
    for j in xrange(len(caractViviendas)):
        
        aux = ""
        palabraAct = caractViviendas[j]
        
        if( palabraAct in x ):
            indice = x.index( palabraAct )
            if(palabraAct == 'monoamb'):caract = '1'
            else: caract = x[indice-1]
            break;
            
        else:
            for ind in xrange( len(x) ):
                if( palabraAct in x[ind] ):
                    if(palabraAct == 'monoamb'):caract = '1'
                    else: caract = x[ind-1]
                    break;
    
    if(caract.isdigit()):
        return int(caract)
    return None  

In [11]:
caracteristicas['ambientes'] = caracteristicas.apply(chequearAmbientes, axis=1)

In [12]:
caracteristicas.head(10)

,id,description,title,properati_url,rooms,ambientes
168728,77816f8811f96453c3a990de643fcf2d2659ac1b,SE TRATA DE UNA VIVIENDA MULTI FAMILIAR CONSIS...,Casa - Merlo,http://www.properati.com.ar/1bqtk_venta_casa_m...,NaN,NaN
168730,bed12005637a003d531a4dcc3aa5a1076787d3b1,Impecable dpto 2 amb. a estrenar con detalles ...,Departamento - San Fernando,http://www.properati.com.ar/1bqto_venta_depart...,NaN,2.0
168731,fb0007abeb74caed5c693783b617a1d1befa82df,Sergio Villella Inmobiliaria presenta departam...,Ov. Lagos 600 - Venta departamento monoambiente.,http://www.properati.com.ar/1bqtq_venta_depart...,NaN,1.0
168733,877ca74c858bc85dd3373eb265795c7fc3afc1e7,"LINDO DUPLEX DE 4 AMB: 3 DORM AMPLIOS, 2 BAÑOS...",Casa en Florida Belgrano-Oeste,http://www.properati.com.ar/1bqts_venta_casa_f...,3.0,4.0
168734,ba7121c00740ba13447d21cda96286cead1615b7,PH AL FRENTE CON ENTRADA INDEPENDIENTE Y ESPAC...,PH 3 AMB. APTO CRÉDITO - Moron,http://www.properati.com.ar/1bqtt_venta_ph_mor...,4.0,3.0
168735,997b2405ded6af49f75ebe01ff168cfb82291223,Departamento de 2 ambientes a estrenar en comp...,Departamento - Condominios Intercontinental,http://www.properati.com.ar/1bqtu_venta_depart...,NaN,2.0
168736,53162225dcf5fe8202e46f95aad3d6183f8cfe9d,IMPORTANTE PROPIEDAD DESARROLLADA EN SOLO UNA ...,Casa - La Plata,http://www.properati.com.ar/1bqtv_venta_casa_l...,NaN,NaN
168737,31dcf5ef4a9af6f1803aa8b998c9be95b223871f,Lindísimo departamento a Estrenar: está ubic...,BELLISIMO 2 AMBIENTES CON GRAN TERRAZA Y COCHERA.,http://www.properati.com.ar/1bqtx_venta_depart...,NaN,2.0
168738,1bcd00c5319313fd70463337fce4d89bb73ebbbc,Winterra S.A.Catamarca 1150CMCPSI5015,Departamento Lagoon Pilar Sector Serena!,http://www.properati.com.ar/1bqty_venta_depart...,NaN,NaN
168739,08acdc231bd21c964a16a8af510c4bd444bb3f77,"Casa de 1 dormitorio, cocina comedor y baño. T...",Casa - El Bolson,http://www.properati.com.ar/1bqu0_venta_casa_s...,1.0,NaN


# Por ultimo vamos a buscar en internet cuando no haya ninguno de los 2 datos.

In [13]:
caracteristicas['webp'] = pd.Series(None, index=caracteristicas.index)

In [14]:
def chequearURL(x):
    
    if( pd.isnull(x[3]) ): return None
    if( not pd.isnull(x[4]) or not pd.isnull(x[5]) ): return None
    
    return buscarURL(x[3])
        
def buscarURL(x):
    
    req = Request(x)
    try:
        resp = urlopen(req, timeout=10)
    except URLError:
        print "Bad URL or timeout"
        return None

    resp_page = resp.read()
    
    return resp_page

In [15]:
caracteristicas['webp'] = caracteristicas.apply(chequearURL, axis=1)

In [16]:
caracteristicas.head(10)

,id,description,title,properati_url,rooms,ambientes,webp
168728,77816f8811f96453c3a990de643fcf2d2659ac1b,SE TRATA DE UNA VIVIENDA MULTI FAMILIAR CONSIS...,Casa - Merlo,http://www.properati.com.ar/1bqtk_venta_casa_m...,NaN,NaN,<!DOCTYPE html>\n<!--[if lt IE 7 ]> <html clas...
168730,bed12005637a003d531a4dcc3aa5a1076787d3b1,Impecable dpto 2 amb. a estrenar con detalles ...,Departamento - San Fernando,http://www.properati.com.ar/1bqto_venta_depart...,NaN,2.0,None
168731,fb0007abeb74caed5c693783b617a1d1befa82df,Sergio Villella Inmobiliaria presenta departam...,Ov. Lagos 600 - Venta departamento monoambiente.,http://www.properati.com.ar/1bqtq_venta_depart...,NaN,1.0,None
168733,877ca74c858bc85dd3373eb265795c7fc3afc1e7,"LINDO DUPLEX DE 4 AMB: 3 DORM AMPLIOS, 2 BAÑOS...",Casa en Florida Belgrano-Oeste,http://www.properati.com.ar/1bqts_venta_casa_f...,3.0,4.0,None
168734,ba7121c00740ba13447d21cda96286cead1615b7,PH AL FRENTE CON ENTRADA INDEPENDIENTE Y ESPAC...,PH 3 AMB. APTO CRÉDITO - Moron,http://www.properati.com.ar/1bqtt_venta_ph_mor...,4.0,3.0,None
168735,997b2405ded6af49f75ebe01ff168cfb82291223,Departamento de 2 ambientes a estrenar en comp...,Departamento - Condominios Intercontinental,http://www.properati.com.ar/1bqtu_venta_depart...,NaN,2.0,None
168736,53162225dcf5fe8202e46f95aad3d6183f8cfe9d,IMPORTANTE PROPIEDAD DESARROLLADA EN SOLO UNA ...,Casa - La Plata,http://www.properati.com.ar/1bqtv_venta_casa_l...,NaN,NaN,<!DOCTYPE html>\n<!--[if lt IE 7 ]> <html clas...
168737,31dcf5ef4a9af6f1803aa8b998c9be95b223871f,Lindísimo departamento a Estrenar: está ubic...,BELLISIMO 2 AMBIENTES CON GRAN TERRAZA Y COCHERA.,http://www.properati.com.ar/1bqtx_venta_depart...,NaN,2.0,None
168738,1bcd00c5319313fd70463337fce4d89bb73ebbbc,Winterra S.A.Catamarca 1150CMCPSI5015,Departamento Lagoon Pilar Sector Serena!,http://www.properati.com.ar/1bqty_venta_depart...,NaN,NaN,<!DOCTYPE html>\n<!--[if lt IE 7 ]> <html clas...
168739,08acdc231bd21c964a16a8af510c4bd444bb3f77,"Casa de 1 dormitorio, cocina comedor y baño. T...",Casa - El Bolson,http://www.properati.com.ar/1bqu0_venta_casa_s...,1.0,NaN,None


In [17]:
aux = caracteristicas[ pd.isnull(caracteristicas['webp']) == True ]
aux.count()

id               11340
description      11340
title            11340
properati_url    11340
rooms             6564
ambientes         5593
webp                 0
dtype: int64

In [18]:
pickle.dump(caracteristicas,open("save12.p","wb"))